<a href="https://colab.research.google.com/github/campu1992/Classification_Conversation_Chats-/blob/main/carga_conversaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from joblib import load
import os
import torch
import numpy as np
import string
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

model = load('/content/drive/MyDrive/Prediccion_clasificacion_chat/modelo_chats.pkl')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Prediccion_clasificacion_chat/fine_tuned_model')
model2 = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Prediccion_clasificacion_chat/fine_tuned_model')

classifier = pipeline('text-classification', model=model2, tokenizer=tokenizer)

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    clean_text = ' '.join(tokens)
    return clean_text

labels = ["venta_de_producto_fisico","venta_de_servicio","venta_adicional","venta_cruzada","venta_de_suscripcion","renovacion_de_contrato","mejora_de_servicio","reduccion_de_servicio","soporte_tecnico","soporte_de_facturacion","procesamiento_de_reembolso","resolucion_de_quejas","solicitud_de_informacion","programacion_de_citas","recoleccion_de_feedback","gestion_de_cuentas","soporte_postventa","conversion_de_clientes_potenciales","recordatorio_de_renovacion","retencion_de_clientes"]

def clasificar_conversacion(conversacion):
    inputs = tokenizer(conversacion, return_tensors="pt", padding=True, truncation=True)
    outputs = model2(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    predicted_label = labels[predicted_class]
    return predicted_label

def predecir_negociacion(conversacion):
    preprocessed_conversacion = preprocess_text(conversacion)
    prediction = model.predict([preprocessed_conversacion])
    return bool(prediction[0])

def entrada_texto():
    conversacion = input("Ingresa la conversación:\n")
    resultado = predecir_negociacion(conversacion)
    resultado2 = clasificar_conversacion(conversacion)
    print(f"¿La negociación fue cerrada exitosamente? : {resultado}")
    print(f"Tipo de negociación: {resultado2}")

# Función para manejar la carga de un archivo CSV
def carga_archivo():
    file_path = input("Ingresa la ruta del archivo CSV:\n")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        conversacion = df['Conversación'].iloc[0]
        resultado = predecir_negociacion(conversacion)
        resultado2 = clasificar_conversacion(conversacion)
        print(f"\n{'-'*50}")
        print(f"¿La negociación fue cerrada exitosamente? : {resultado}")
        print(f"Tipo de negociación: {resultado2}")
        print(f"{'-'*50}\n")
    else:
        print("El archivo no existe. Por favor verifica la ruta.")

# Función principal que maneja el menú y la lógica del flujo
def main():
    while True:
        print("Selecciona una opción:")
        print("1. Ingresar conversación manualmente")
        print("2. Cargar archivo CSV")
        print("3. Salir")
        choice = input("Opción: ")

        if choice == '1':
            entrada_texto()
        elif choice == '2':
            carga_archivo()
        elif choice == '3':
            break
        else:
            print("Opción no válida. Por favor intenta nuevamente.")

        repeat = input("¿Deseas cargar otra conversación? (s/n): ")
        if repeat.lower() != 's':
            break

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Selecciona una opción:
1. Ingresar conversación manualmente
2. Cargar archivo CSV
3. Salir
Opción: 1
Ingresa la conversación:
Agente (Laura): ¡Hola! ¿En qué puedo ayudarte hoy?  Cliente (Carlos): Hola, estoy buscando un nuevo smartphone. Mi viejo ya está fallando y necesito uno con una buena cámara y batería duradera.  Agente (Laura): Entiendo, Carlos. Tenemos una amplia gama de smartphones. ¿Tienes alguna marca en mente o alguna característica específica que estés buscando además de una buena cámara y batería?  Cliente (Carlos): He escuchado cosas buenas sobre los modelos de Samsung. También quiero que tenga suficiente espacio de almacenamiento porque tomo muchas fotos y videos.  Agente (Laura): Perfecto, Samsung es una excelente elección. Te recomendaría el Samsung Galaxy S21. Tiene una cámara de 64MP, batería de 4000mAh y viene con 128GB de almacenamiento interno. ¿Qué te parece?  Cliente (Carlos): Suena bien. ¿Cuánto cuesta?  Agente (Laura): Actualmente, el Samsung Galaxy S21 está 